In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS, Chroma
from langchain.document_loaders import DirectoryLoader
import chromadb
from langchain.embeddings import OllamaEmbeddings

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [24]:
# RAG prompt
from langchain import hub

QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-llama")

In [15]:
llm = Ollama(
    model="llama2",
    verbose=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="llama2")

In [18]:
def process_and_embed_docs(dir_path, model):
    chroma_instance = chromadb.Client()
    doc_loader = CSVLoader(dir_path)
    loaded_docs = doc_loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    split_docs = splitter.split_documents(loaded_docs)
    print("Embedding document...")
    database = Chroma.from_documents(documents=split_docs, embedding=model)
    print("Embedding and chunking process completed successfully")
    return database

In [21]:
database = process_and_embed_docs("../data/tarot_readings.csv", oembed)
retriever = database.as_retriever()

Embedding document...
Embedding and chunking process completed successfully


In [29]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [38]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.chains import LLMChain

model = llm
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a very good tarot reader. You like to give short answers",
        ),
        ("human", "{question}"),
    ]
)

In [36]:
chain = LLMChain(llm=model, prompt=prompt, output_parser=StrOutputParser())

In [37]:
chain.run(question="I draw The Sun tarot card. What does it mean?")


Ah, The Sun card! This is a wonderful sign, my dear. It symbolizes joy, happiness, and contentment. You're feeling fulfilled and satisfied with your life right now, and that's wonderful. Embrace this feeling and let it fill you up. It's a great time to pursue your passions and interests, and you may find yourself feeling more creative and inspired than ever before. Just remember to stay grounded and not get too caught up in your own happiness - there will always be ups and downs, after all! 😊

"\nAh, The Sun card! This is a wonderful sign, my dear. It symbolizes joy, happiness, and contentment. You're feeling fulfilled and satisfied with your life right now, and that's wonderful. Embrace this feeling and let it fill you up. It's a great time to pursue your passions and interests, and you may find yourself feeling more creative and inspired than ever before. Just remember to stay grounded and not get too caught up in your own happiness - there will always be ups and downs, after all! 😊"

In [39]:
chain.run(question="I draw The Sun, Nine of Pentacles, and Two of Cups cards. What does it mean?")


Ah, an excellent combination of cards! Here's what they might be indicating:

The Sun card represents joy, success, and fulfillment in your life. You're experiencing happiness and contentment, and things are going well for you. This card can also indicate good luck and positive energy flowing into your life.

Nine of Pentacles: This card suggests that you're feeling confident and capable in your abilities. You're in a state of personal growth and development, and you're learning to value yourself and your worth. It can also indicate financial success and stability.

Two of Cups: This card represents harmony and balance in relationships. You're in a place of mutual understanding and respect with those around you, and you're feeling a strong connection to others. It can also indicate a romantic or platonic relationship that's blossoming.

Overall, these cards suggest that you're experiencing happiness, success, and positive energy in your life, particularly in your relationships. Keep u

"\nAh, an excellent combination of cards! Here's what they might be indicating:\n\nThe Sun card represents joy, success, and fulfillment in your life. You're experiencing happiness and contentment, and things are going well for you. This card can also indicate good luck and positive energy flowing into your life.\n\nNine of Pentacles: This card suggests that you're feeling confident and capable in your abilities. You're in a state of personal growth and development, and you're learning to value yourself and your worth. It can also indicate financial success and stability.\n\nTwo of Cups: This card represents harmony and balance in relationships. You're in a place of mutual understanding and respect with those around you, and you're feeling a strong connection to others. It can also indicate a romantic or platonic relationship that's blossoming.\n\nOverall, these cards suggest that you're experiencing happiness, success, and positive energy in your life, particularly in your relationshi

In [42]:
chain.run(question="Tell me more about it")

Ah, I see! Well, my dear, the tarot reading reveals that you are currently on a path of great transformation and growth. It's a time of change and evolution for you, both personally and professionally. You may be feeling a bit unsettled or uncertain about the future, but trust that you are exactly where you need to be. Embrace this journey with an open heart and mind, and know that it will lead you to new and exciting opportunities. 💫✨

"Ah, I see! Well, my dear, the tarot reading reveals that you are currently on a path of great transformation and growth. It's a time of change and evolution for you, both personally and professionally. You may be feeling a bit unsettled or uncertain about the future, but trust that you are exactly where you need to be. Embrace this journey with an open heart and mind, and know that it will lead you to new and exciting opportunities. 💫✨"

In [40]:
database = process_and_embed_docs("../data/tarot_readings.csv", oembed)
retriever = database.as_retriever()

Embedding document...


KeyboardInterrupt: 